In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()


In [3]:
df = spark.read.csv("data/case.csv", header=True, inferSchema=True)
df.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [4]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [6]:
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

In [7]:
df.explain()

== Physical Plan ==
*(1) Project [case_id#16, case_opened_date#17, case_closed_date#18, SLA_due_date#19 AS case_due_date#201, case_late#20, num_days_late#21, case_closed#22, dept_division#23, service_request_type#24, SLA_days#25, case_status#26, source_id#27, request_address#28, council_district#29]
+- FileScan csv [case_id#16,case_opened_date#17,case_closed_date#18,SLA_due_date#19,case_late#20,num_days_late#21,case_closed#22,dept_division#23,service_request_type#24,SLA_days#25,case_status#26,source_id#27,request_address#28,council_district#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/matthewknight/codeup-data-science/spark-exercises/data/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [8]:
df.groupBy('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [10]:
df = df.withColumn('case_late', expr('case_late == "Yes"'))

In [11]:
df = df.withColumn('case_closed', expr('case_closed == "Yes"'))

In [12]:
df.show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | false                
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [13]:
df.select(col('council_district'), format_string('%03d', col('council_district'))).show()

+----------------+-------------------------------------+
|council_district|format_string(%03d, council_district)|
+----------------+-------------------------------------+
|               5|                                  005|
|               3|                                  003|
|               3|                                  003|
|               3|                                  003|
|               7|                                  007|
|               7|                                  007|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|             

In [14]:
df = df.withColumn("council_district", col("council_district").cast("string"))


In [16]:
df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)
   

In [17]:
df.show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | false                
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [18]:
fmt = "M/d/yy H:mm"
df.select(
    'case_opened_date',
    to_timestamp('case_opened_date', fmt)
).show(5)

+----------------+-----------------------------------------------+
|case_opened_date|to_timestamp(`case_opened_date`, 'M/d/yy H:mm')|
+----------------+-----------------------------------------------+
|     1/1/18 0:42|                            2018-01-01 00:42:00|
|     1/1/18 0:46|                            2018-01-01 00:46:00|
|     1/1/18 0:48|                            2018-01-01 00:48:00|
|     1/1/18 1:29|                            2018-01-01 01:29:00|
|     1/1/18 1:34|                            2018-01-01 01:34:00|
+----------------+-----------------------------------------------+
only showing top 5 rows



In [19]:
df = df.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))
df = df.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))
df = df.withColumn('case_due_date', to_timestamp('case_due_date', fmt))

In [20]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)



In [23]:
df.select(
    col('request_address'),
    lower(trim('request_address'))
).show(truncate=False)

+----------------------------------------+----------------------------------------+
|request_address                         |lower(trim(request_address))            |
+----------------------------------------+----------------------------------------+
|2315  EL PASO ST, San Antonio, 78207    |2315  el paso st, san antonio, 78207    |
|2215  GOLIAD RD, San Antonio, 78223     |2215  goliad rd, san antonio, 78223     |
|102  PALFREY ST W, San Antonio, 78223   |102  palfrey st w, san antonio, 78223   |
|114  LA GARDE ST, San Antonio, 78223    |114  la garde st, san antonio, 78223    |
|734  CLEARVIEW DR, San Antonio, 78228   |734  clearview dr, san antonio, 78228   |
|BANDERA RD and BRESNAHAN                |bandera rd and bresnahan                |
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio,

In [24]:
df = df.withColumn("request_address", trim(lower(df.request_address)))


In [25]:
df = df.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))


In [26]:
df.show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | false                
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 case_due_date        | 2018-01-05 08:30:00  
 case_late            | false     

In [31]:
# case_lifetime
# -if case is closed then diff between close and open dates -- days_to_close
# -else diff between open date and now -- case_age

df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [33]:
df

DataFrame[case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: string, zipcode: string, case_age: int, days_to_closed: int, case_lifetime: int]

In [35]:
dept = spark.read.csv('data/dept.csv', header=True)

In [37]:
df = df.join(dept, 'dept_division', 'left')

In [38]:
train, test = df.randomSplit([.8, .2])

In [39]:
train.count(), test.count()

(673140, 168564)